import statements and data you need

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline


In [12]:
# dataframe for first csv

# Path to the data - select the path that works for you 
file_path = '../data-mining-semester-project/data/Census race.csv'

# Load the data into a DataFrame 
dfCensus = pd.read_csv(file_path)

dfCensus = dfCensus.dropna()
dfCensus= dfCensus.replace('!!',' ', regex=True)
# dfCensus.info()
dfCensus.head()

,GEO_ID,NAME,P1_001N,P1_002N,P1_003N,P1_004N,P1_005N,P1_006N,P1_007N,P1_008N,...,P1_062N,P1_063N,P1_064N,P1_065N,P1_066N,P1_067N,P1_068N,P1_069N,P1_070N,P1_071N
0,id,Geographic Area Name,Total:,Total: Population of one race:,Total: Population of one race: White alone,Total: Population of one race: Black or Afri...,Total: Population of one race: American Indi...,Total: Population of one race: Asian alone,Total: Population of one race: Native Hawaii...,Total: Population of one race: Some Other Ra...,...,Total: Population of two or more races: Popu...,Total: Population of two or more races: Popu...,Total: Population of two or more races: Popu...,Total: Population of two or more races: Popu...,Total: Population of two or more races: Popu...,Total: Population of two or more races: Popu...,Total: Population of two or more races: Popu...,Total: Population of two or more races: Popu...,Total: Population of two or more races: Popu...,Total: Population of two or more races: Popu...
1,0500000US01001,"Autauga County, Alabama",58805,55648,42160,11445,217,881,35,910,...,0,2,2,0,0,0,0,0,0,0
2,0500000US01003,"Baldwin County, Alabama",231767,216743,189399,18217,1582,2067,143,5335,...,0,0,0,0,0,0,0,0,0,0
3,0500000US01005,"Barbour County, Alabama",25223,24523,11317,11933,116,117,1,1039,...,0,2,2,0,0,0,0,0,0,0
4,0500000US01007,"Bibb County, Alabama",22293,21534,16555,4413,60,32,9,465,...,0,0,0,0,0,0,0,0,0,0


In [86]:
# dataframe for second csv file 
file_path2 = '../data-mining-semester-project/data/Unemployment.csv'

# Load the data into a DataFrame 
dfUnemployment = pd.read_csv(file_path2)
dfUnemployment = dfUnemployment.dropna()
# getting rid of values outside of 2015-2020

dfUnemployment = dfUnemployment.drop(["Civilian_labor_force_2000"],1)
dfUnemployment= dfUnemployment.drop(["Employed_2000"],1)
dfUnemployment= dfUnemployment.drop(["Unemployed_2000"],1)
dfUnemployment = dfUnemployment.drop(["Unemployment_rate_2000"],1)
dfUnemployment = dfUnemployment.drop(["Civilian_labor_force_2001"],1)
dfUnemployment= dfUnemployment.drop(["Employed_2001"],1)
dfUnemployment= dfUnemployment.drop(["Unemployed_2001"],1)
dfUnemployment = dfUnemployment.drop(["Unemployment_rate_2001"],1)

dfUnemployment = dfUnemployment.drop(["Civilian_labor_force_2002"],1)
dfUnemployment= dfUnemployment.drop(["Employed_2002"],1)
dfUnemployment= dfUnemployment.drop(["Unemployed_2002"],1)
dfUnemployment = dfUnemployment.drop(["Unemployment_rate_2002"],1)


dfUnemployment = dfUnemployment.drop(["Civilian_labor_force_2003"],1)
dfUnemployment= dfUnemployment.drop(["Employed_2003"],1)
dfUnemployment= dfUnemployment.drop(["Unemployed_2003"],1)
dfUnemployment = dfUnemployment.drop(["Unemployment_rate_2003"],1)

dfUnemployment = dfUnemployment.drop(["Civilian_labor_force_2004"],1)
dfUnemployment= dfUnemployment.drop(["Employed_2004"],1)
dfUnemployment= dfUnemployment.drop(["Unemployed_2004"],1)
dfUnemployment = dfUnemployment.drop(["Unemployment_rate_2004"],1)

dfUnemployment = dfUnemployment.drop(["Civilian_labor_force_2005"],1)
dfUnemployment= dfUnemployment.drop(["Employed_2005"],1)
dfUnemployment= dfUnemployment.drop(["Unemployed_2005"],1)
dfUnemployment = dfUnemployment.drop(["Unemployment_rate_2005"],1)

dfUnemployment = dfUnemployment.drop(["Civilian_labor_force_2006"],1)
dfUnemployment= dfUnemployment.drop(["Employed_2006"],1)
dfUnemployment= dfUnemployment.drop(["Unemployed_2006"],1)
dfUnemployment = dfUnemployment.drop(["Unemployment_rate_2006"],1)

dfUnemployment = dfUnemployment.drop(["Civilian_labor_force_2007"],1)
dfUnemployment= dfUnemployment.drop(["Employed_2007"],1)
dfUnemployment= dfUnemployment.drop(["Unemployed_2007"],1)
dfUnemployment = dfUnemployment.drop(["Unemployment_rate_2007"],1)

dfUnemployment = dfUnemployment.drop(["Civilian_labor_force_2008"],1)
dfUnemployment= dfUnemployment.drop(["Employed_2008"],1)
dfUnemployment= dfUnemployment.drop(["Unemployed_2008"],1)
dfUnemployment = dfUnemployment.drop(["Unemployment_rate_2008"],1)


dfUnemployment = dfUnemployment.drop(["Civilian_labor_force_2009"],1)
dfUnemployment= dfUnemployment.drop(["Employed_2009"],1)
dfUnemployment= dfUnemployment.drop(["Unemployed_2009"],1)
dfUnemployment = dfUnemployment.drop(["Unemployment_rate_2009"],1)

dfUnemployment = dfUnemployment.drop(["Civilian_labor_force_2010"],1)
dfUnemployment= dfUnemployment.drop(["Employed_2010"],1)
dfUnemployment= dfUnemployment.drop(["Unemployed_2010"],1)
dfUnemployment = dfUnemployment.drop(["Unemployment_rate_2010"],1)


dfUnemployment = dfUnemployment.drop(["Civilian_labor_force_2011"],1)
dfUnemployment= dfUnemployment.drop(["Employed_2011"],1)
dfUnemployment= dfUnemployment.drop(["Unemployed_2011"],1)
dfUnemployment = dfUnemployment.drop(["Unemployment_rate_2011"],1)

dfUnemployment = dfUnemployment.drop(["Civilian_labor_force_2012"],1)
dfUnemployment= dfUnemployment.drop(["Employed_2012"],1)
dfUnemployment= dfUnemployment.drop(["Unemployed_2012"],1)
dfUnemployment = dfUnemployment.drop(["Unemployment_rate_2012"],1)

dfUnemployment = dfUnemployment.drop(["Civilian_labor_force_2013"],1)
dfUnemployment= dfUnemployment.drop(["Employed_2013"],1)
dfUnemployment= dfUnemployment.drop(["Unemployed_2013"],1)
dfUnemployment = dfUnemployment.drop(["Unemployment_rate_2013"],1)

dfUnemployment = dfUnemployment.drop(["Civilian_labor_force_2014"],1)
dfUnemployment= dfUnemployment.drop(["Employed_2014"],1)
dfUnemployment= dfUnemployment.drop(["Unemployed_2014"],1)
dfUnemployment = dfUnemployment.drop(["Unemployment_rate_2014"],1)

dfUnemployment = dfUnemployment.drop(["Rural_urban_continuum_code_2013"],1)
dfUnemployment= dfUnemployment.drop(["Urban_influence_code_2013"],1)
dfUnemployment= dfUnemployment.drop(["Metro_2013"],1)

dfUnemployment= dfUnemployment.drop(["Median_Household_Income_2019"],1)
dfUnemployment= dfUnemployment.drop(["Med_HH_Income_Percent_of_State_Total_2019"],1)
  

# dfUnemployment = dfUnemployment.drop(dfUnemployment[dfUnemployment.loc["State"]=="AK"])

# drop anything not in Alabama, Arkansas, Delaware, the District of Columbia, Florida, Georgia, Kentucky, Louisiana, Maryland, Mississippi, North Carolina, Oklahoma, South Carolina, Tennessee, Texas, Virginia, and West Virginia.



dfUnemployment.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3129 entries, 2 to 3195
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   FIPS_Code                  3129 non-null   int64  
 1   State                      3129 non-null   object 
 2   Area_name                  3129 non-null   object 
 3   Civilian_labor_force_2015  3129 non-null   object 
 4   Employed_2015              3129 non-null   object 
 5   Unemployed_2015            3129 non-null   object 
 6   Unemployment_rate_2015     3129 non-null   float64
 7   Civilian_labor_force_2016  3129 non-null   object 
 8   Employed_2016              3129 non-null   object 
 9   Unemployed_2016            3129 non-null   object 
 10  Unemployment_rate_2016     3129 non-null   float64
 11  Civilian_labor_force_2017  3129 non-null   object 
 12  Employed_2017              3129 non-null   object 
 13  Unemployed_2017            3129 non-null   objec

In [92]:
# VSRR_Provisional_Drug_Overdose_Death_Counts.csv
file_path3 = '../data-mining-semester-project/data/VSRR_Provisional_Drug_Overdose_Death_Counts.csv'

# Load the data into a DataFrame 
dfODeath = pd.read_csv(file_path3)
# dfODeath = dfODeath.dropna()
dfODeath = dfODeath.drop(["Footnote"],1)
dfODeath = dfODeath.drop(["Footnote Symbol"],1)
dfODeath = dfODeath.drop(["State Name"],1)

# dfODeath = dfODeath.drop([dfODeath["State"]=="AK"],1)

# need to drop anything that is not Alabama, Arkansas, Delaware, the District of Columbia, Florida, Georgia, Kentucky, Louisiana, Maryland, Mississippi, North Carolina, Oklahoma, South Carolina, Tennessee, Texas, Virginia, and West Virginia.
# 
dfODeath.head()


,State,Year,Month,Period,Indicator,Data Value,Percent Complete,Percent Pending Investigation,Predicted Value
0,AK,2015,April,12 month-ending,"Natural, semi-synthetic, & synthetic opioids, ...",NaN,100,0.0,NaN
1,AK,2015,April,12 month-ending,Natural & semi-synthetic opioids (T40.2),NaN,100,0.0,NaN
2,AK,2015,April,12 month-ending,"Natural & semi-synthetic opioids, incl. methad...",NaN,100,0.0,NaN
3,AK,2015,April,12 month-ending,Number of Deaths,"4,133",100,0.0,NaN
4,AK,2015,April,12 month-ending,"Opioids (T40.0-T40.4,T40.6)",NaN,100,0.0,NaN


In [15]:
# dataframe for fourth csv file 

# Underlying Cause of Death, 1999-2019.csv
file_path4 = '../data-mining-semester-project/data/Underlying Cause of Death, 1999-2019.csv'

# drop anything not in Alabama, Arkansas, Delaware, the District of Columbia, Florida, Georgia, Kentucky, Louisiana, Maryland, Mississippi, North Carolina, Oklahoma, South Carolina, Tennessee, Texas, Virginia, and West Virginia.
# reduce time to 2015 to 2020 for this dataset 
# Load the data into a DataFrame 
dfCDeath = pd.read_csv(file_path4)
dfCDeath.head()
# dfCDeath.info()


,County,County Code,Drug/Alcohol Induced,Drug/Alcohol Induced Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,Population,Crude Rate,Crude Rate Lower 95% Confidence Interval,Crude Rate Upper 95% Confidence Interval,Crude Rate Standard Error,Age Adjusted Rate,Age Adjusted Rate Lower 95% Confidence Interval,Age Adjusted Rate Upper 95% Confidence Interval,% of Total Deaths
0,"Autauga County, AL",1001,All other non-drug and non-alcohol causes,O,All other non-drug and non-alcohol causes,O9,536.0,55869.0,959.4,878.2,1040.6,41.4,812.2,742.7,881.7,0.0%
1,"Baldwin County, AL",1003,Drug-induced causes,D,Drug poisonings (overdose) Unintentional (X40-...,D1,26.0,223234.0,11.6,7.6,17.1,2.3,13,8.3,19.3,0.0%
2,"Baldwin County, AL",1003,Alcohol-induced causes,A,All other alcohol-induced causes,A9,27.0,223234.0,12.1,8.0,17.6,2.3,10.8,6.9,16.0,0.0%
3,"Baldwin County, AL",1003,All other non-drug and non-alcohol causes,O,All other non-drug and non-alcohol causes,O9,2222.0,223234.0,995.4,954.0,1036.8,21.1,721.3,690.2,752.3,0.1%
4,"Barbour County, AL",1005,All other non-drug and non-alcohol causes,O,All other non-drug and non-alcohol causes,O9,330.0,24686.0,1336.8,1192.6,1481.0,73.6,1021.4,907.8,1135.0,0.0%


In [18]:
# state death data covid19
file_path5 = '../data-mining-semester-project/data/statesCOVID.csv'

dfStateC19Death = pd.read_csv(file_path5)
dfStateC19Death.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [9]:
#county death data covid19
file_path6 = '../data-mining-semester-project/data/us-counties.csv'
# drop anything not a southern state 
dfCountyC19Death = pd.read_csv(file_path6)
dfCountyC19Death.head()

,date,geoid,county,state,cases,cases_avg,cases_avg_per_100k,deaths,deaths_avg,deaths_avg_per_100k
0,2020-01-21,USA-53061,Snohomish,Washington,1,0.14,0.02,0,0.0,0.0
1,2020-01-22,USA-53061,Snohomish,Washington,0,0.14,0.02,0,0.0,0.0
2,2020-01-23,USA-53061,Snohomish,Washington,0,0.14,0.02,0,0.0,0.0
3,2020-01-24,USA-53061,Snohomish,Washington,0,0.14,0.02,0,0.0,0.0
4,2020-01-24,USA-17031,Cook,Illinois,1,0.14,0.00,0,0.0,0.0


In [91]:
# death data for type of drug 
# VSRR_Provisional_Drug_Overdose_Death_Counts.csv
file_path4 = '../data-mining-semester-project/data/VSRR_Provisional_Drug_Overdose_Death_Counts-3.csv'

# drop anything not in a southern state to start, retain data from 2015 to end of 2020

# Load the data into a DataFrame 
dfTypeDrug = pd.read_csv(file_path3)
# dfTypeDrug=dfTypeDrug.dropna(subset =["Predicted Value"],).copy()
dfTypeDrug = dfTypeDrug.drop(["State Name"],1)
dfTypeDrug = dfTypeDrug.drop(["Footnote"],1)
dfTypeDrug = dfTypeDrug.drop(["Footnote Symbol"],1)
# dfState = dfTypeDrug["State"]
# # dfState = np.array(dfState)
# print(dfState)
dfTypeDrug.head()
# dfTypeDrug.head()

,State,Year,Month,Period,Indicator,Data Value,Percent Complete,Percent Pending Investigation,Predicted Value
0,AK,2015,April,12 month-ending,"Natural, semi-synthetic, & synthetic opioids, ...",NaN,100,0.0,NaN
1,AK,2015,April,12 month-ending,Natural & semi-synthetic opioids (T40.2),NaN,100,0.0,NaN
2,AK,2015,April,12 month-ending,"Natural & semi-synthetic opioids, incl. methad...",NaN,100,0.0,NaN
3,AK,2015,April,12 month-ending,Number of Deaths,"4,133",100,0.0,NaN
4,AK,2015,April,12 month-ending,"Opioids (T40.0-T40.4,T40.6)",NaN,100,0.0,NaN
